In [1]:
pip install --upgrade google-cloud-storage


     ------------------------------------ 110.2/110.2 KB 636.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [1]:
import os
from google.cloud import bigquery

<h3> Upload files from computer to google Bigquery

In [1]:
from pathlib import Path
import time
import os
from google.cloud import bigquery
# this stuff is important
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='cloud_key1.json'

def table_reference(project_id, dataset_id, table_id):
    dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
    table_ref = bigquery.TableReference(dataset_ref, table_id)
    return table_ref


def delete_dataset_tables(project_id, dataset_id):
    tables = client.list_tables(f'{project_id}.{dataset_id}')
    for table in tables:
        client.delete_table(table)
    print('Tables deleted.')

def upload_csv(client, table_ref, csv_file):
    client.delete_table(table_ref, not_found_ok=True)

    load_job_configuration1 = bigquery.LoadJobConfig()
    # load_job_configuration.schema = [
    #     bigquery.SchemaField('movieid', 'INTEGER', mode='REQUIRED'),
    #     bigquery.SchemaField('title', 'STRING', mode='NULLABLE'),
    #     bigquery.SchemaField('genres', 'STRING', mode='NULLABLE')
    # ]

    load_job_configuration1.autodetect = True
    load_job_configuration1.source_format = bigquery.SourceFormat.CSV
    # load_job_configuration1.skip_leading_rows = 1
    load_job_configuration1.allow_quoted_newlines = True

    with open(csv_file, 'rb') as source_file:
        upload_job = client.load_table_from_file(
            source_file,
            destination=table_ref,          
            location='US',
            job_config=load_job_configuration1
        )

    while upload_job.state != 'DONE':
        time.sleep(2)
        upload_job.reload()
        print(upload_job.state)
    print(upload_job.result())
# files in  a particular directory on my system
d=os.path.expanduser("~/Desktop/").replace("\\",'/')
project_id = 'my-project-356323'
dataset_id = 'upload_testdata'
for files in os.listdir(d+"gcloud/data"):
    client = bigquery.Client()
    table_name=files.strip('.csv')
    csv_file = d+"gcloud/data"+'/'+files
    table_ref = table_reference(project_id, dataset_id, table_name)
    upload_csv(client, table_ref, csv_file)
    print()

# delete_dataset_tables(project_id, dataset_id)     

DONE
LoadJob<project=my-project-356323, location=US, id=09e8585f-ee8f-4c2a-891a-371f08deab42>

DONE
LoadJob<project=my-project-356323, location=US, id=d48820be-9883-434f-b572-bf285100028b>



<h3> Upload dataframes to google Bigquery

In [2]:
import pandas as pd
from google.cloud import bigquery

# Set up the BigQuery client and table reference
d=os.path.expanduser("~/Desktop/").replace("\\",'/')
project_id = 'my-project-356323'
dataset_id = 'upload_testdata'
df=pd.read_csv('movies.csv')

client = bigquery.Client()

# Set up the table

table_id = 'my_table'
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref)

# Create the table in BigQuery
table = client.create_table(table)
print(f'Table {table.table_id} created.')

# Load the DataFrame into the table
job = client.load_table_from_dataframe(df, table_ref, location='US')

# Wait for the load job to complete
job.result()
print(f'Loaded {job.output_rows} rows into {table_id}.')

# for files in os.listdir(d+"gcloud/data"):
#     client = bigquery.Client()
#     table_name=files.strip('.csv')
#     csv_file = d+"gcloud/data"+'/'+files
#     table_ref = table_reference(project_id, dataset_id, table_name)
#     upload_csv(client, table_ref, csv_file)
#     print()

Table my_table created.
Loaded 34208 rows into my_table.
